In [40]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("datasets_stocks/intraday_jpn_iexg.csv",index_col=0)

In [3]:
data

,open,high,low,last,close,volume,date,symbol,exchange
0,28.050,28.060,27.820,27.950,28.750,16601.0,2020-11-30T20:00:00+0000,HMC,IEXG
1,4.435,4.435,4.320,4.325,4.490,8632.0,2020-11-30T19:00:00+0000,MUFG,IEXG
2,93.610,93.980,92.760,93.625,94.070,16927.0,2020-11-30T19:00:00+0000,SNE,IEXG
3,28.050,28.060,27.820,27.905,28.750,10040.0,2020-11-30T18:00:00+0000,HMC,IEXG
4,135.870,135.870,134.655,135.110,140.160,5920.0,2020-11-30T18:00:00+0000,TM,IEXG
...,...,...,...,...,...,...,...,...,...
4995,28.320,28.320,28.320,NaN,28.320,NaN,2019-12-07T02:00:00+0000,CAJ,IEXG
4996,5.445,5.445,5.435,NaN,5.445,NaN,2019-12-07T01:00:00+0000,MUFG,IEXG
4997,28.330,28.330,28.320,NaN,28.320,NaN,2019-12-07T01:00:00+0000,CAJ,IEXG
4998,5.445,5.450,5.445,NaN,5.445,NaN,2019-12-07T00:00:00+0000,MUFG,IEXG


In [88]:
stock = "SNE" # Sony

df = data[data["symbol"] == stock].set_index("date")

In [89]:
df

,open,high,low,last,close,volume,symbol,exchange
date,,,,,,,,
2020-11-30T19:00:00+0000,93.610,93.980,92.760,93.625,94.07,16927.0,SNE,IEXG
2020-11-30T17:00:00+0000,93.610,93.980,92.760,93.140,94.07,10166.0,SNE,IEXG
2020-11-30T16:00:00+0000,93.610,93.980,92.760,93.040,94.07,6925.0,SNE,IEXG
2020-11-30T15:00:00+0000,93.610,93.980,93.490,93.930,94.07,2983.0,SNE,IEXG
2020-11-28T02:00:00+0000,93.980,94.500,93.820,94.070,93.23,666878.0,SNE,IEXG
...,...,...,...,...,...,...,...,...
2020-06-04T16:00:44+0000,66.880,67.385,66.465,66.580,66.79,26556.0,SNE,IEXG
2020-06-04T15:00:36+0000,66.880,67.385,66.870,67.140,66.79,14206.0,SNE,IEXG
2020-06-02T15:00:20+0000,66.165,66.595,65.990,66.400,65.49,22919.0,SNE,IEXG


In [30]:
stocks = list(data.symbol.unique())
[*stocks,"Date"]

['HMC',
 'MUFG',
 'SNE',
 'TM',
 'SMFG',
 'CAJ',
 'MFG',
 'TYHOF',
 'CHUEF',
 'TOELY',
 'HTHIY',
 'MARUY',
 'PCRFY',
 'SFTBY',
 'SMTOF',
 'Date']

In [98]:
# annualised daily variance

# 1. calculate j_t: diff between closing on day t-1 and opening on day t
opens = pd.read_csv("datasets_stocks/japan_open.csv")
closes = pd.read_csv("datasets_stocks/japan_adj_close.csv")

opens = opens[[stock,"Date"]].set_index("Date")
closes = closes[[stock,"Date"]].set_index("Date")

# j_t = 100*(ln(P_t_open) - ln(P_{t-1}_close))
j = 100*np.log(opens.iloc[1:] / closes.shift(1).iloc[1:])
j = j[j.index <= "2020-12-01"] # truncate
j.head()

,SNE
Date,
2020-01-03,-0.437282
2020-01-06,0.626304
2020-01-07,2.382426
2020-01-08,1.227986
2020-01-09,1.702179


In [139]:
# 2. continuous return within a day
# r_{i,t} = 100*(ln(P_{i,t}) - ln(P_{i,t-1}))
# we want R_t = sum(r_{i,t}^2)

df.index = pd.DatetimeIndex(df.index)
dates_avail = sorted(list(set(df.index.date)),reverse=True) # <= unique dates available in data
R = pd.DataFrame(index=dates_avail, columns=["R"])

for date in dates_avail:
    prices = df[df.index.date == date].copy()["close"] # prices observed on this date
    rs = 100*np.log(prices.iloc[1:]/prices.shift(1).iloc[1:])# all the rs calculated on this date
    R.loc[date]["R"] = sum(rs**2)
R.index = pd.DatetimeIndex(R.index)   
R.head()

,R
2020-11-30,0
2020-11-28,0
2020-11-27,0
2020-11-26,0
2020-11-25,1.25119


In [141]:
# 3. Annualised daily variance RV (return variance)
# RV_t = 252*(j_t^2 + R_t)

merged = pd.merge(j, R, how="inner", left_index=True, right_index=True)
merged.columns = ["j", "R"]

merged["RV"] = 252*(merged["j"]**2 + merged["R"])

In [142]:
merged

,j,R,RV
2020-11-30,-0.447474,0,50.4587
2020-11-27,0.801244,0,161.782
2020-11-25,2.823204,1.25119,2323.86
2020-11-24,1.393522,0.044682,500.62
2020-11-23,0.133269,0,4.47566
...,...,...,...
2020-06-09,-0.175175,3.38577,860.947
2020-06-08,0.281882,0,20.0233
2020-06-05,0.602595,0.841818,303.645
2020-06-04,0.313924,0,24.8341
